# System Identification Guide: ParameterKalmanFilterAgent

This notebook provides a guide to the `ParameterKalmanFilterAgent`. This agent uses a **Kalman Filter (KF)**, a more statistically rigorous approach, to perform online parameter estimation. 

## 1. Theoretical Background

The Kalman Filter is an optimal estimator for linear systems with Gaussian noise. It operates in a two-step **predict-update** cycle. While it is famously used for state estimation (e.g., tracking a moving object), it can be adapted for parameter estimation.

The key insight is to treat the unknown parameters of the system as the **state** to be estimated. The process model assumes the parameters are mostly constant (but allows for some random change, modeled by `process_noise_Q`), and the measurement model describes how those parameters relate the system's inputs and outputs.

Like the `RLSAgent`, this implementation is designed for a first-order system:

`y(k) = a₁ * y(k-1) + b₁ * u(k-1)`

The KF approach is powerful because it explicitly models both **process noise** (the possibility that the true parameters might slowly change) and **measurement noise** (the inaccuracy of our sensors).

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.control.kf_estimator import ParameterKalmanFilterAgent

## 2. Code Example: Estimating Parameters with KF

We will use the exact same virtual system as in the RLS tutorial to allow for a direct comparison. We will also add some noise to the measurements to demonstrate the KF's ability to handle it.

In [ ]:
# 1. Define the "True" System and the KF Agent
true_a1 = 0.95
true_b1 = 0.5
measurement_noise_std_dev = 0.1

kf_agent = ParameterKalmanFilterAgent(
    initial_params={"a1": 0.0, "b1": 0.0}, # Start with no knowledge
    process_noise_Q=1e-5, # Assume parameters are very stable but might change slightly
    measurement_noise_R=measurement_noise_std_dev**2 # Variance of the measurement noise
)

# 2. Simulation Setup
n_steps = 100
u_signal = np.random.rand(n_steps) * 10
y_signal = np.zeros(n_steps)
y_prev = 0
u_prev = 0

history = {
    "a1_estimate": [],
    "b1_estimate": []
}

# 3. Simulation Loop
for k in range(n_steps):
    # Simulate the "true" system
    true_y = true_a1 * y_prev + true_b1 * u_prev
    # Add noise to create a realistic sensor measurement
    measured_y = true_y + np.random.normal(0, measurement_noise_std_dev)
    
    # Feed the current input and the *noisy* output to the KF agent
    kf_agent.step(inflow=u_signal[k], observed_outflow=measured_y)
    
    # Get the agent's current best guess
    current_estimates = kf_agent.get_state()
    
    # Record history
    history["a1_estimate"].append(current_estimates['a1'])
    history["b1_estimate"].append(current_estimates['b1'])
    
    # Update state for the next iteration (using the true value)
    y_prev = true_y
    u_prev = u_signal[k]

print("Simulation complete.")
print(f"Final a1 estimate: {history['a1_estimate'][-1]:.4f} (True value: {true_a1})")
print(f"Final b1 estimate: {history['b1_estimate'][-1]:.4f} (True value: {true_b1})")

## 4. Visualization

Plotting the estimated parameters over time shows the Kalman Filter's convergence.

In [ ]:
time_axis = np.arange(n_steps)

plt.figure(figsize=(12, 6))
plt.plot(time_axis, history['a1_estimate'], label='a1 Estimate (KF)')
plt.axhline(y=true_a1, color='r', linestyle='--', label=f'True a1 = {true_a1}')
plt.plot(time_axis, history['b1_estimate'], label='b1 Estimate (KF)')
plt.axhline(y=true_b1, color='g', linestyle='--', label=f'True b1 = {true_b1}')

plt.title('Kalman Filter Parameter Estimation Over Time')
plt.xlabel('Time Step (k)')
plt.ylabel('Parameter Value')
plt.legend()
plt.grid(True)
plt.ylim(0, 1.2)
plt.show()

Despite the noisy measurements, the plot shows that the Kalman Filter still effectively learns the system's true parameters, converging smoothly towards the correct values. The tuning of `process_noise_Q` and `measurement_noise_R` is key to balancing the filter's responsiveness to new data against its ability to reject noise.